In [17]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel

In [74]:
import chardet

# Detect the encoding of the file
file_path = 'machine.txt'

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

encoding = result['encoding']
print(f"Detected encoding: {encoding}")

# Read the file with the detected encoding
data = pd.read_csv(file_path, sep='\t', encoding=encoding )
data.head()


Detected encoding: UTF-16


,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5


In [85]:
data2 = pd.read_csv('pokemon.csv')
data2.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [88]:
from scipy import stats

print(data)

# Perform one-tailed t-test
print(data.columns)
t_stat, p_value = stats.ttest_ind(data['New machine'], data['    Old machine'], alternative='less')

# Define the significance level (alpha) and adjust for one-tailed test
alpha = 0.05
alpha_adjusted = alpha / 2

print("P_value is ", round(p_value, 5))
# Check if the p-value is less than the adjusted significance level
if p_value < alpha_adjusted:
    print("There is sufficient evidence.")
else:
    print("There is not sufficient evidence.")


   New machine      Old machine
0         42.1             42.7
1         41.0             43.6
2         41.3             43.8
3         41.8             43.3
4         42.4             42.5
5         42.8             43.5
6         43.2             43.1
7         42.3             41.7
8         41.8             44.0
9         42.7             44.1
Index(['New machine', '    Old machine'], dtype='object')
P_value is  0.00161
There is sufficient evidence.


In [89]:
sample =data2.sample(30, random_state = 1)
sample['difference'] = sample['Attack']-sample['Defense']
sample.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,difference
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,26
510,460,Abomasnow,Grass,Ice,494,90,92,75,92,85,60,4,False,17
175,161,Sentret,Normal,NaN,215,35,46,34,35,45,20,2,False,12
735,667,Litleo,Fire,Normal,369,62,50,58,73,54,72,6,False,-8
242,224,Octillery,Water,NaN,480,75,105,75,105,75,45,2,False,30


In [90]:
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(2.8, 36.32079294288603)

In [91]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: 2.80
The standard deviation of our samples differences is: 36.32
Our t statistics is: 0.42


In [92]:
tc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
tc

2.045229642132703

# Inferential statistics - ANOVA

### Part 1

### Part 2

In [50]:
excel_file_path = 'anova_lab_data.xlsx'

# Read the data from the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)

df

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [51]:
print(df.columns)


Index(['Power ', 'Etching Rate'], dtype='object')


In [52]:
group_df = df.groupby('Power ')['Etching Rate'].agg(Power_mean='mean',Samples='size').reset_index()
group_df

,Power,Power_mean,Samples
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


In [54]:
S2t = 0
for power in df['Power '].unique():
    ng = len(df[df['Power '] == power])
    S2t  += ng * ( ( df[df['Power '] == power]['Etching Rate'].mean() - df['Etching Rate'].mean() ) ** 2)
S2t /= ( df['Power '].nunique() - 1 ) 

print("The value of S2t is {:.2f}".format(S2t))

The value of S2t is 9.09


In [61]:
S2E = 0
for power in df['Power '].unique():
    for rate in df[df['Power '] == power]['Etching Rate']:
        S2E += ( rate - df[df['Power '] == power]['Etching Rate'].mean() ) ** 2
S2E /= ( len(df) - df['Power '].nunique() )

print()
print("The value of S2E is {:.2f}".format(S2E))


The value of S2E is 0.25


In [65]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [66]:
st.f.cdf(F,dfn=d1, dfd=d2)


0.9999924934157276

In [67]:
1 - st.f.cdf(F,dfn=d1, dfd=d2)


7.5065842723986975e-06